In [1]:
import tika
import pandas as pd
import json
import ast
from AlvinGoogleAPI import *
pd.set_option('display.max_columns', None)

In [3]:
df = pd.read_csv("../data/Bik dataset - papers with endpoint reached.tsv", sep='\t', encoding = "ISO-8859-1")
df = df.dropna(subset=['Authors'])
df.to_csv('../data/Bik dataset.csv',index=False)
df.shape

(214, 17)

In [4]:
df = pd.read_csv('../data/sisi.csv')

In [5]:
df.head()

,Unnamed: 0,Authors,Title,Citation,DOI,Year,0,1,2,3,FINDINGS,Reported,Correction Date,Retraction,Correction,No Action,SUM Completed,institution_info,city_country,LatLng,city_name
0,0,"Inka Regine Weingaertner, Sarah Koutnik, Herma...",Chronic Morphine Treatment Attenuates Cell Gro...,"PLOS One, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053510,01/01/13,1.0,NaN,1.0,NaN,"duplicated blot, shifted microscopy",1,10/09/15,NaN,1.0,NaN,1,"['Institute of Pharmacology, Toxicology and Ph...","[' Ludwig-Maximilians-University', "" Germany'""...","(48.1372, 11.5755)",['munich']
1,1,"Jessica M. Esparza, Eileen O’Toole, Linya Li, ...",Katanin Localization Requires Triplet Microtub...,"PLOS ONE, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053940,01/01/13,NaN,1.0,NaN,NaN,duplicated colony strip,1,01/15/16,NaN,1.0,NaN,1,"['Department of Genetics, Washington Universit...","[' St. Louis', ' Missouri', "" United States of...","(38.6358, -90.2451)",['st. louis']
2,2,"Sreedevi Avasarala, Fangfang Zhang, Guangliang...",Curcumin Modulates the Inflammatory Response a...,"PLOS ONE, February 2013 | Volume 8 | Issue 2 |...",10.1371/journal.pone.0057285,01/01/13,NaN,NaN,1.0,NaN,reuse of one lane in different blot experiment,1,08/04/15,NaN,1.0,NaN,1,"['Department of Oral Biology and Pathology, Sc...","["" United States of America'"", ' New York', ' ...","(40.9099, -73.1213)","['stony brook university', 'stony brook', 'new..."
3,3,"Rounak Nassirpour, Pramod P. Mehta, Sangita M....",miR-221 Promotes Tumorigenesis in Human Triple...,"PLOS ONE, April 2013 | Volume 8 | Issue 4 | e6...",10.1371/journal.pone.0062170,01/01/13,1.0,NaN,NaN,1,Duplicated / mirrored bands in blot,1,04/10/17,1.0,NaN,NaN,1,"['Oncology Research, Pfizer Worldwide Research...","[' California', ' San Diego', "" United States ...","(32.8312, -117.1225)","['san diego', 'california']"
4,4,"Jianuo Liu, Peng Xu, Cory Collins, Han Liu, J...",HIV-1 Tat Protein Increases Microglial Outward...,"PLOS ONE, May 2013 | Volume 8 | Issue 5 | e649...",10.1371/journal.pone.0064904,01/01/13,NaN,1.0,NaN,NaN,multiple duplicated microscopy images,1,09/19/14,NaN,1.0,NaN,1,"['Neurophysiology Laboratory, Department of Ph...","[' Omaha', ' Nebraska', "" United States of Ame...","(41.2627, -96.0522)",['omaha']


In [4]:
found = {}
many = set()
failedgreen = set()
failednear = set()

In [8]:

for index,row in df.iterrows():
    institution_info = row['institution_info']
    
    if type(institution_info) == str:
        try:
            institution_info = ast.literal_eval(institution_info)
            if len(institution_info) ==1:
                institution_info = institution_info[0].split(',')
            else:
                setdata = set(institution_info)
                temp = []
                for i in setdata:
                    temp = temp + i.split(',')
                institution_info = temp
        except:
            institution_info = institution_info.split(',')

    institutions = []
    for institution in institution_info:
        institution = institution.lower().strip()
        if not ('university' in institution or 'college' in institution):
            continue

        institutions.append(institution)

    if len(institutions) == 0:
        df.at[0,'GreenPct'] = 'NA'
        df.at[0,'NearBodyOfWater'] = 'NA'
        continue

    if len(institutions) > 1:
        many.add(i)

    institution = institutions[0]
    if institution in found:
        green, near = found[institution]
    else:
        try:
            # get_image(institution, satellite=True)
            img = cv2.imread(f"satellite_view/{institution}.png")
            green = get_green_pct(img)
        except Exception as e:
            print('sat error', e)
            green = 'NA'
            failedgreen.add(institution)

        try:
            # get_image(institution)
            img = cv2.imread(f"roadmap_view/{institution}.png")
            near = count_water(img)
        except Exception as e:
            print('roadmap error',e)
            near = 'NA'
            failednear.add(institution)

        found[institution] = [green,near]
    # print(index, institution,green,near)

    df.at[index,'University'] = institution
    df.at[index,'GreenPct'] = green
    df.at[index,'NearBodyOfWater'] = near


In [9]:
df.to_csv('../data/Tiger dataset.csv',index=False)

In [3]:
df = pd.read_csv('../data/Tiger dataset.csv')
found = {}

In [4]:
df.head(2)

,Unnamed: 0,Authors,Title,Citation,DOI,Year,0,1,2,3,FINDINGS,Reported,Correction Date,Retraction,Correction,No Action,SUM Completed,institution_info,city_country,LatLng,city_name,GreenPct,NearBodyOfWater,University
0,0,"Inka Regine Weingaertner, Sarah Koutnik, Herma...",Chronic Morphine Treatment Attenuates Cell Gro...,"PLOS One, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053510,01/01/13,1.0,NaN,1.0,NaN,"duplicated blot, shifted microscopy",1,10/09/15,NaN,1.0,NaN,1,"['Institute of Pharmacology, Toxicology and Ph...","[' Ludwig-Maximilians-University', "" Germany'""...","(48.1372, 11.5755)",['munich'],NaN,NaN,ludwig-maximilians-university
1,1,"Jessica M. Esparza, Eileen O’Toole, Linya Li, ...",Katanin Localization Requires Triplet Microtub...,"PLOS ONE, January 2013 | Volume 8 | Issue 1 | ...",10.1371/journal.pone.0053940,01/01/13,NaN,1.0,NaN,NaN,duplicated colony strip,1,01/15/16,NaN,1.0,NaN,1,"['Department of Genetics, Washington Universit...","[' St. Louis', ' Missouri', "" United States of...","(38.6358, -90.2451)",['st. louis'],33.13%,False,washington university school of medicine


In [6]:
for index,row in df.iterrows():
    try:
        institution = row['University'].lower()
    except:
        continue
    
    if institution in found:
        parks = found[institution]
    else:
        try:
            # get_image(institution)
            img = cv2.imread(f"roadmap_view/{institution}.png")
            parks = count_park(img)
        except Exception as e:
            print(index, 'roadmap error',e)
            parks='NA'
        found[institution] = parks
    df.at[index,'NearParks'] = parks

24 roadmap error 'NoneType' object is not iterable
41 roadmap error 'NoneType' object is not iterable
89 roadmap error 'NoneType' object is not iterable


In [8]:
df.to_csv('Tiger dataset.csv',index=False)